In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.nn import sigmoid
from tensorflow.keras.losses import binary_crossentropy



In [ ]:
df = pd.read_csv('recruitment_data.csv')

df.drop(['RecruitmentStrategy'], axis=1,inplace= True)
df['HiringDecision'].value_counts()

In [ ]:
#making lables frequecy same to avoid bias
for HiringDecision , h_df in df.groupby('HiringDecision'):
    if len(h_df) < 400:
        df.drop(h_df.index,axis=0 , inplace= True)
        
    elif len(h_df) > 465:
        x = len(h_df) - 465
        df.drop(h_df[ :x].index, axis= 0 , inplace=True)
        
df

In [ ]:
#normalization
x= df[["Age","EducationLevel","ExperienceYears","PreviousCompanies","DistanceFromCompany","InterviewScore","PersonalityScore","HiringDecision"]]
org_mean , org_std = x.mean() , x.std()

df[["Age","EducationLevel","ExperienceYears","PreviousCompanies","DistanceFromCompany","InterviewScore","PersonalityScore","HiringDecision"]]= ( x - org_mean)/ org_std

targets = df[["HiringDecision"]]
input = df.drop(["HiringDecision"] , axis=1)

input.info()


In [ ]:
# converting into tensors
inputs_tf=tf.constant(input, dtype= tf.float64)
targets_tf = tf.constant(targets , dtype= tf.float64)

In [ ]:
#initializing the mode
weights = tf.Variable(tf.random.normal(shape=(9,1),dtype= tf.float64))
bias = tf.Variable(tf.random.normal(shape=(1,),dtype= tf.float64)) 


In [ ]:
#main training loop 
lr = 0.001
mean_losses = []   
epochs = range(1000)
for epochs in tqdm(epochs):
 
    with tf.GradientTape() as tape:
        #making prediction
        # preds = tf.add(inputs_tf , weights , bias)
        preds = tf.add(tf.matmul(inputs_tf , weights), bias)
        prob= sigmoid(preds)
        
        #calculation loss 
        loss = binary_crossentropy(targets_tf, prob)
        mean_loss = tf.reduce_sum(loss)/len(targets_tf)
        
        print(f" for {epochs} epochs ,  mean loss is {mean_loss} ")
        #taking gradiants
        w_grad , b_grad = tape.gradient(mean_loss, [ weights, bias])
        
        #updating model
        weights = tf.Variable(weights - lr * w_grad)
        bias = tf.Variable(bias - lr * b_grad)


print(mean_losses)
plt.plot(epochs, mean_losses)
plt.xlabel('Epochs')
plt.ylabel('Mean Loss')
plt.title('Mean Loss over Epochs')
plt.show()



    

In [ ]:
print(mean_losses.shape)
# print(epochs.shape)

In [ ]:
correct = 0 
predictions = inputs_tf @ weights + bias
prob = sigmoid(predictions) 

for i in range(len(prob)):
    if prob[i] > 0.5: 
        if int (targets_tf[i]) == 1: 
           correct += 1      

    else:
        if int(targets_tf[i]) == 0:
            correct +=1
            
correct/len(prob)




In [ ]:
# Calculate accuracy
predictions = tf.round(prob)
correct_predictions = tf.equal(predictions, targets_tf)
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float64))

print(f"Epoch {epochs}: Accuracy = {accuracy.numpy() * 100:.2f}%")